In [1]:
import pandas as pd
import json
import os
from datetime import date

# Collecting the data from Twitter

In [2]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])


In [3]:
#username = 'templivs'
#username_list  = ['GilbertCollard','dav_dec','Carbongate','bcassoret',
#                  'Electroversenet','thinkfree55', 'KlassLib','sauvonsleclimat']

username_list = ['francisrichard','MazdaArtaxerxes','templivs','prontipronto',
                'Chabadalala','cocktail2Funk','HopitalC',
                'riva_vitale','Remifasol57','AitiDouze', 'QAnonAustria1', 'gotteswerk2411']

swiss_accounts = ['KlimaschutzCH', 'GrueneCH', 'proclimCH', 'EperonP', 'MathiasTemujin',
                  'klimastreik', 'AlimEquitables', 'ProNaturaSuisse', 'vertliberaux', 'Munsterma',
                  'bourg_d', 'LesVertsSuisses', 'ClimatSuisse', 'gpsuisse', 'IliasPanchard', 'ATE_Suisse']

immigration = ['Kalvingrad1291','democratesuisse','VigilanceIslam','lioneljonson01','ChWilhou',
               'HunterSThomson2','A_Addams_','ObservateursCH','JuanCandida','novopress']

username_list = immigration
# create the path to save the experiment indexed with a date
today = date.today()
date_string = today.strftime("%y%m%d")
print("date string =", date_string)

#date_string = '191128'

data_path = 'immigration_reduced' + date_string+ '/'
#get_tweets = python_tweets.get_user_timeline(screen_name = username,  
#                                           count = 200, include_rts = True)
if not os.path.isdir(data_path):
    os.mkdir(data_path)
    print('Path created:',data_path)

date string = 191216
Path created: immigration_reduced191216/


In [4]:
import pysad

In [5]:
thres = 2 # minimal number of mentions to keep
max_day_old = 2 # number max of days in the past

In [6]:
def process_user_list(python_tweets, data_path, username, thres=3, max_day_old=None):
    users_dic = {'username':[], 'Nb_diff_mentions': []}
    print('Collecting the tweets for the last {} days.'.format(max_day_old))
    new_users_list = []
    for user in username_list:
        mentions = create_user_edgelist(python_tweets, data_path, user, thres=thres, max_day_old=max_day_old)
        if not mentions.empty:
            users_mentioned = mentions['mention'][mentions['weight']>thres]
            #users_mentioned = users_mentioned.unique() # not sure this is useful
            new_users_list += users_mentioned.tolist()
        users_dic['username'].append(user)
        users_dic['Nb_diff_mentions'].append(len(mentions))
    users_df = pd.DataFrame(users_dic)
    return new_users_list,users_df

In [7]:

def create_user_edgelist(python_tweets, data_path, username, thres, max_day_old):
	# Process the user username and its mentioned users
	# save in a file the edgelist for the user and each mentioned user

	# initial user
	print('Processing',username)
	#try:
	mention_grouped,mgl = pysad.collect_user_mention(username,python_tweets,data_path, max_day_old=max_day_old)
	#except:
	#    print('exception catched on user {} !!!!!!!!!!!!'.format(username))
	#    return
	if mention_grouped.empty:
		print('Empty tweet list. Processing stopped for user ',username)
		return mention_grouped
	mentionfilename = data_path + username + '_mentions' +'_t' +str(thres)+'.csv'
	print('Writing {} tweets in {}.'.format(len(mention_grouped),mentionfilename))
	mention_grouped.to_csv(mentionfilename)
	#nb_mentions = len(mention_grouped)
	#print('User {} done. Nb different mentions: {}'.format(username,nb_mentions))
	return mention_grouped

In [8]:
users_dic = {'username':[], 'Nb_mentions': [], 'mentions_of_mentions': []}
print('Collecting the tweets for the last {} days.'.format(max_day_old))
exploration_depth = 2
total_username_list = username_list
for depth in range(exploration_depth):
    print('')
    print('******* Processing users at {}-hop distance *******'.format(depth))
    new_users_list,users_df = process_user_list(python_tweets, data_path, username_list, thres=thres, max_day_old=max_day_old)
    #New users to collect:
    username_list = list(set(new_users_list).difference(set(total_username_list))) # remove the one already collected
    total_username_list += username_list



******* Processing users at 0-hop distance *******
Processing Kalvingrad1291
Empty tweet list. Processing stopped for user  Kalvingrad1291
Processing democratesuisse
Empty tweet list. Processing stopped for user  democratesuisse
Processing VigilanceIslam
Empty tweet list. Processing stopped for user  VigilanceIslam
Processing lioneljonson01
Writing 9 tweets in immigration_reduced191216/lioneljonson01_mentions_t2.csv.
Processing ChWilhou
Writing 8 tweets in immigration_reduced191216/ChWilhou_mentions_t2.csv.
Processing HunterSThomson2
Writing 75 tweets in immigration_reduced191216/HunterSThomson2_mentions_t2.csv.
Processing A_Addams_
Writing 9 tweets in immigration_reduced191216/A_Addams__mentions_t2.csv.
Processing ObservateursCH
Empty tweet list. Processing stopped for user  ObservateursCH
Processing JuanCandida
Empty tweet list. Processing stopped for user  JuanCandida
Processing novopress
Empty tweet list. Processing stopped for user  novopress

******* Processing users at 1-hop di

In [9]:
print('Total number of users collected:')
print(len(total_username_list),len(set(total_username_list)))

Total number of users collected:
102 102


# Loading the saved data into an edge table

In [10]:
import glob

edge_df = pd.DataFrame()
for filename in glob.glob(data_path + '*_mentions' +'_t' +str(thres)+ '.csv'):
    new_edge_df = pd.read_csv(filename)
    print('{} with {} tweets.'.format(filename,len(new_edge_df)))
    edge_df = edge_df.append(new_edge_df)
    

immigration_reduced191216/A_Addams__mentions_t2.csv with 9 tweets.
immigration_reduced191216/S_o_l_s_t_i_c_e_mentions_t2.csv with 38 tweets.
immigration_reduced191216/PsykoSauce_mentions_t2.csv with 2 tweets.
immigration_reduced191216/SainteOraora_mentions_t2.csv with 81 tweets.
immigration_reduced191216/KimJongUnique_mentions_t2.csv with 27 tweets.
immigration_reduced191216/TimeMachine1980_mentions_t2.csv with 75 tweets.
immigration_reduced191216/PaulFortune1975_mentions_t2.csv with 21 tweets.
immigration_reduced191216/T_Nenninger_mentions_t2.csv with 66 tweets.
immigration_reduced191216/holy_Phoenixx_mentions_t2.csv with 16 tweets.
immigration_reduced191216/gregtabibian_mentions_t2.csv with 7 tweets.
immigration_reduced191216/raumnitz_mentions_t2.csv with 16 tweets.
immigration_reduced191216/dr_l_alexandre_mentions_t2.csv with 6 tweets.
immigration_reduced191216/JulienPain_mentions_t2.csv with 6 tweets.
immigration_reduced191216/lioneljonson01_mentions_t2.csv with 9 tweets.
immigrati

In [11]:
# display edges with number of hashtags >1
edge_df[edge_df['hashtags'].apply(lambda x : len(x.split()))>1]

,Unnamed: 0,user,mention,weight,hashtags,date,urls
0,0,A_Addams_,AllisonLeeSolin,1,"['cdnpoli', 'debatimmigration']",['2019-12-14 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
1,1,A_Addams_,Babs_cat33,1,"['blacktwitter', 'racism']",['2019-12-15 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
2,2,A_Addams_,InstitutILIADE,1,"['blacktwitter', 'racism']",['2019-12-15 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
3,3,A_Addams_,RenaudCamus,1,"['blacktwitter', 'racism']",['2019-12-15 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
4,4,A_Addams_,RevueInAnalysis,1,"['cdnpoli', 'debatimmigration']",['2019-12-14 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
5,5,A_Addams_,TheeAndroidRulz,1,"['nrc', 'americadivided']",['2019-12-15 11:21:03'],['https://paper.li/A_Addams_/1421662602?editio...
6,6,A_Addams_,aires_aston,1,"['nrc', 'americadivided']",['2019-12-15 11:21:03'],['https://paper.li/A_Addams_/1421662602?editio...
7,7,A_Addams_,rick_hoyer,1,"['cdnpoli', 'debatimmigration']",['2019-12-14 23:21:01'],['https://paper.li/A_Addams_/1421662602?editio...
8,8,A_Addams_,truckster1,1,"['nrc', 'americadivided']",['2019-12-15 11:21:03'],['https://paper.li/A_Addams_/1421662602?editio...
4,4,S_o_l_s_t_i_c_e,Chris_A10_USA,1,"['Ireland', 'migrant', 'insanity']",['2019-12-15 21:25:47'],[]


In [12]:
import networkx as nx


def graph_from_edgeslist(edge_df,degree_min):
    print('Creating the graph fro th edge list')
    G = nx.from_pandas_edgelist(edge_df,source='user',target='mention', edge_attr=['weight','hashtags','date','urls'])
    print('Nb of nodes:',G.number_of_nodes())
    # Drop
    remove = [node for node,degree in dict(G.degree()).items() if degree < degree_min]
    G.remove_nodes_from(remove)
    print('Nb of nodes after removing less connected nodes:',G.number_of_nodes())
    return G

In [14]:
DEGREE_MIN = 3 # Minimal number of connections in the graph

G = graph_from_edgeslist(edge_df,DEGREE_MIN)
isolates = list(nx.isolates(G))
G.remove_nodes_from(isolates)
print('removed {} isolated nodes.'.format(len(isolates)))

Creating the graph fro th edge list
Nb of nodes: 431
Nb of nodes after removing less connected nodes: 38
removed 1 isolated nodes.


In [15]:
G.is_directed()

False

## Community detection

In [16]:
import community

In [17]:
#first compute the best partition
partition = community.best_partition(G)
nx.set_node_attributes(G,partition,name='community')
nb_partitions = max(partition.values())+1
print('Nb of partitions:',nb_partitions)

Nb of partitions: 4


In [18]:
graphname = 'multiusersgraph'
#graphname = 'GBRgraph'

In [19]:
# Save the graph
graphfilename = data_path + graphname + '_t' + str(thres) +'_graph.gexf'
nx.write_gexf(G,graphfilename)
print('Wrote',graphfilename)

Wrote immigration_reduced191216/multiusersgraph_t2_graph.gexf


# Hashtags, dates and urls
Hashtags, dates and urls are on the edges of the network.
We can get the most common hashtags within a community and also betwenn communities using the edges that connect them.

In [20]:
import ast # convert string to list
from collections import Counter
import datetime
import numpy as np

In [21]:
# get the hashtags for each community and inter-communities
tags_dic = {}
dates_dic = {}
url_dic = {}
for node1,node2,data in G.edges(data=True):
    if node1 == node2:
        print('Self edge',node1)
    n1_com = G.nodes[node1]['community']
    n2_com = G.nodes[node2]['community']
    new_key = str(n1_com) + '-' + str(n2_com) # intra / inter community code
    # Convert string to list
    x = ast.literal_eval(data['hashtags'])
    d = ast.literal_eval(data['date'])
    u = ast.literal_eval(data['urls'])
    keywords = [n.strip() for n in x]
    date_list = [n.strip() for n in d]
    urls = [n.strip() for n in u]
    if new_key not in tags_dic:
        tags_dic[new_key] = keywords
        dates_dic[new_key] = date_list
        url_dic[new_key] = urls
    else:
        tags_dic[new_key] += keywords 
        dates_dic[new_key] += date_list
        url_dic[new_key] += urls

In [22]:
# Display the most common hashtags in communities and inter communities
#for key in tags_dic:
#    most_common = Counter(tags_dic[key]).most_common(5)
#    print(key)
#    print(most_common)

In [23]:
def compute_meantime(date_list):
    d_list = [ datetime.datetime.strptime(dt,'%Y-%m-%d %H:%M:%S') for dt in date_list]
    second_list = [x.timestamp() for x in d_list]
    meand = np.mean(second_list)
    stdd = np.std(second_list)
    return datetime.datetime.fromtimestamp(meand),datetime.timedelta(seconds=stdd)

In [24]:
# Create a table with time and hashtags for each community
comm_list = []
for key in np.arange(nb_partitions):
    keykey = str(key)+ '-' +str(key)
    most_common = Counter(tags_dic[keykey]).most_common(5)
    meandate,stddate = compute_meantime(dates_dic[keykey])
    #print('Community',key)
    #print(most_common)
    #print('Average date: {} and std deviation: {} days'.format(meandate.date(),stddate.days))
    comm_dic = {'Community':key, 'Average date':meandate.date(), 'Deviation (days)':stddate.days}
    for htag_nb in range(5): # filling the table with the hashtags
        if htag_nb < len(most_common):
            comm_dic['hashtag'+str(htag_nb)] = most_common[htag_nb][0]
        else:
            comm_dic['hashtag'+str(htag_nb)] = ''
    comm_list.append(comm_dic)
community_table = pd.DataFrame(comm_list)

In [25]:
community_table

,Community,Average date,Deviation (days),hashtag0,hashtag1,hashtag2,hashtag3,hashtag4
0,0,2019-12-15,0,Belgique,Berchem,MissFrance,Toulouse,crèche
1,1,2019-12-15,0,interview,misogyne,moustache,GretaIceberg,MeToo
2,2,2019-12-15,0,MissFrance,,,,
3,3,2019-12-15,0,,,,,


### Process the urls

In [26]:
# Dataframe with the url of each cluster and inter-clusters
urltocomm = []
for key in url_dic:
    for url in url_dic[key]:
        urltocomm.append([url,key,1])
url_table = pd.DataFrame(urltocomm, columns=['url','Community','Occurence'])
url_table = url_table.groupby(['url','Community']).agg(Occurence=('Occurence',sum))
url_table = url_table.reset_index()

In [27]:
# Replace all bit.ly url by the correct one
import requests

session = requests.Session()  # so connections are recycled

for index, row in url_table.iterrows():
    url = row['url']
    if 'bit.ly' in url:
        resp = session.head(url, allow_redirects=True)
        url_table.loc[index,'url'] = resp.url
        #print(resp.url)

In [28]:
# Drop the references to twitter web site
twitterrowindices = url_table[url_table['url'].str.contains('twitter.com')].index
filtered_url_table = url_table.drop(twitterrowindices)

In [29]:
# Keep only the intra community links
dropindices = []
for index, row in filtered_url_table.iterrows():
    if row['Community'][0] != row['Community'][-1]:
        dropindices.append(index)
    else: # modify the entry
        filtered_url_table.loc[index,'Community'] = row['Community'][0]
filtered_url_table = filtered_url_table.drop(dropindices)
filtered_url_table.reset_index(inplace=True)
filtered_url_table.drop(columns=['index'], inplace=True)

# Sort users by community and save in a excel sheet

In [30]:
# Sort users by community and store their node degree (importance)
community_nodes = {}
for node,data in G.nodes(data=True):
    community_nb = data['community']
    if  community_nb not in community_nodes:
        community_nodes[community_nb] = [(node, G.degree(node))]
    else:
        community_nodes[community_nb].append((node, G.degree(node)))


# Display the exmaple of community c_idx
#c_idx = 0
#ddf = pd.DataFrame(community_nodes[c_idx],columns=['User','Degree'])
#print('list of most connected users in community',c_idx)
#ddf.sort_values(by='Degree',ascending=False).head(20)

In [31]:
# Save to an excel file
with pd.ExcelWriter(data_path + 'graph_infos.xlsx') as writer:
    for community_nb in community_nodes:
        ddf = pd.DataFrame(community_nodes[community_nb],columns=['User','Degree'])
        ddf = ddf.sort_values(by='Degree',ascending=False)#.head(20)
        ddf.to_excel(writer, sheet_name='Community_' + str(community_nb),index=False)
    community_table.to_excel(writer, sheet_name='Hashtags',index=False)
    users_df.to_excel(writer, sheet_name='Initial_users_details',index=False)
    filtered_url_table.to_excel(writer, sheet_name='List_of_urls',index=False)
    # Set the column width
    column_width = 25
    for sheet in writer.sheets: 
        worksheet = writer.sheets[sheet]
        for col in ['A','B','C','D','E','F','G','H']:
            worksheet.column_dimensions[col].width = column_width
    writer.sheets['List_of_urls'].column_dimensions['A'].width = 100